# Predicting from an HMM in split data

In certain cases, you may not want to use the built-in cross-validation (CV) functionality (covered in [*Predicting from an HMM*](./Prediction_example.ipynb)). This may be because you are using a different scheme to define your folds, you want to train and test on separate populations (like patients and controls), or you are using training/testing as separate steps in another routine. This notebook shows how to use the toolbox's functionality to do this. 

If you are new to the (GL)HMM or are unsure how to fit the (GL)HMM, start with the tutorials [*Standard Gaussian Hidden Markov Model*](./GaussianHMM_example.ipynb) or [*Gaussian-Linear Hidden Markov Model*](./GLHMM_example.ipynb). 

**NOTE: Running this tutorial requires data from the Human Connectome Project (HCP). Make sure you have been registered to use the HCP data. If not, register and download the HCP data by following the instructions on the [Human Connectome Project's website](https://www.humanconnectome.org/study/hcp-young-adult/data-use-terms).**

We use the HCP S1200 Young Adult dataset ([van Essen et al., 2013](https://pubmed.ncbi.nlm.nih.gov/23684880/)) to fit the HMM and predict age and cognitive variables, as well as classify sex from resting-state fMRI dynamics (amplitude and functional connectivity). For reproducibility and since the dataset is very large, we provide a pretrained HMM in the example data folder. 

Authors: Christine Ahrends <christine.ahrends@cfin.au.dk>

## Outline
1. [Preparation](#preparation)
    * [Load and prepare data](#load-data)
    * [Train HMM](#train-hmm)
        * [Levels of separation between training and test set](#train-test-sep)
2. [Example: Predicting individual traits from an HMM in split data](#example-prediction)
3. [Example: Classifying sex from an HMM in split data](#example-classification)

## Preparation <a id="preparation"></a>
If you dont have the **GLHMM-package** installed, then run the following command in your terminal:

```pip install glhmm```

**Import libraries**\
Let's start by importing the required libraries and modules.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from glhmm import glhmm, preproc, prediction, io, graphics

### Load and prepare data <a id="load-data"></a>
This requires that the following files are available:
* data: HCP rest fMRI timeseries from 1001 subjects in groupICA50 parcellation
* behav: behavioural/demographic items from 1001 HCP subjects
* T_t: indices for beginning and start of each subject's scanning session
* twins: matrix indicating family structure (subjects x subjects), zeros for unrelated subjects and positive values for related subjects (diagonal will be ignored)
* confounds: confounding variables for 1001 subjects (here sex and head motion)

In [ ]:
# load data from csv files and convert to numpy arrays
data = pd.read_csv('tc1001_RESTall_groupICA50.csv', header=None).to_numpy()
T_t = pd.read_csv('T.csv', header=None).to_numpy()
behav = pd.read_csv('behav1001_35var.csv', header=None).to_numpy()
twins = pd.read_csv('twins.csv', header=None).to_numpy()
confounds = pd.read_csv('confounds.csv', header=None).to_numpy()

# check that dimensions of input files are correct:
print(data.shape) # data should be (n_subjects*n_timepoints, n_parcels)
print(behav.shape) # behav should be (n_subjects, n_variables)
print(T_t.shape) # T_t should be (n_subjects, 2)
print(twins.shape) # twins should be (n_subjects, n_subjects)
print(confounds.shape) # confounds should be (n_subjects, n_confounds) or (n_subjects,)

Standardise timeseries for all following computations. This is an important step, especially when looking at differences between individuals, to make sure that predictions are not driven by measurement noise.

In [ ]:
data_preproc,_ = preproc.preprocess_data(data, T_t)
del data # we will only use the standardised version of the time series going forward

### Split dataset into training and test set
To illustrate a split dataset, we will here simply use the first 100 subjects as training set and the next 50 subjects as a test set. In reality, these may be obtained from splitting functions, e.g. sklearn's [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split), or come from separate populations or datasets.

In [ ]:
train_indices = np.arange(100)
test_indices = np.arange(100, 150)

# separate time series for training and test set
T_train = T_t[train_indices]
T_test = T_t[test_indices]
# to get time series indices, use aux functions:
ts_train = glhmm.auxiliary.make_indices_from_T(T_train)
ts_test = glhmm.auxiliary.make_indices_from_T(T_test)
data_train = data_preproc[ts_train,:]
data_test = data_preproc[ts_test,:]

# separate target variables for training and test set
target_train = behav[train_indices,:]
target_test = behav[test_indices,:]

# separate confounds for training and test set
confounds_train = confounds[train_indices, :]
confounds_test = confounds[test_indices, :]

# separate group structure for training and test set 
twins_train = twins[train_indices, train_indices]
twins_test = twins[test_indices, test_indices]

### Train HMM <a id="train-hmm"></a>
#### Levels of separation between training and test set <a id="train-test-sep"></a>
When working with split data, it is important to consider on which levels of data processing this divide needs to be preserved. In our case, these levels are: preprocessing (not covered here), fitting the group-level HMM, feature normalisation/deconfounding, and model training. While training and test set are always kept separate for out-of-sample prediction for the last step, the other steps may differ. The gold-standard recommendation is to keep training and test data separate on every level to avoid leakage of information. However, in cases where the purpose is mainly to explain relations in the data (rather than generalising predictions to unseen data), the divide may be more important for the target variable. By default, the toolbox functions keep training and test set separate for feature normalisation/deconfounding, and model training. Whether the group-level HMM is fit only to the training dataset or to the concatenated data of training and test set is up to the user. 

We will here show an example of training the HMM only on the subjects we want to later use as training set to follow the gold-standard recommendation. Note though that when using the Fisher kernel, heterogeneity between the training and test set may bias the estimation, since individual subjects' features are here defined *in reference to the group-level model*, which test subjects would then not be part of (see [Ahrends, Woolrich, & Vidaurre, 2024](https://elifesciences.org/reviewed-preprints/95125) for an in-depth exploration of the effects).

In [ ]:
%%capture
hmm = glhmm.glhmm(model_beta='no', K=6, covtype='full')
hmm.train(X=None, Y=data_train, indices=T_train)

Alternatively, you can load the pre-trained HMM (which was fitted to all subjects).

In [ ]:
# hmm = io.load_hmm('./example_data/hmm_hcp_preproc.pkl')

## Example: Predicting individual traits from an HMM in split data <a id="example-prediction"></a>

## Example: Classifying sex from an HMM In split data <a id="example-classification"></a>